#Resampling

Think of resampling like taste-testing food before serving a big party:

1. You make small samples from your big batch

2. You taste each sample to see if it's good

3. This helps you predict how the entire batch will taste

In statistics: **Resampling means repeatedly drawing samples from your data to understand how your model performs.**

## Why do we need Resampling

- Withoug Resampling:
1. Wit fit model once on all data
2. We have no idea how it performs on new data

- With Resampling:
1. We simulate having new data by splitting what we have
2. We get multiple performance estimates
3. We understand how reliable our model is

### Simple Problem

When we train a model, we usually measure **training error**.

But what we actually care about is **test error**:

> How well will this model perform on *new data*?

#### Key Issue

* Training error is **optimistic** (too good)
* Test error is **unknown** (we don’t have infinite new data)


### Solution

Use **resampling methods**:

* Repeatedly split data
* Train models
* Measure errors on data not used for training

---

#### Training Error vs Test Error

| Type           | Meaning                             |
| -------------- | ----------------------------------- |
| Training Error | Error on data used to fit the model |
| Test Error     | Error on new, unseen data           |

> Training error almost always **underestimates** test error.

---


Two main types we'll learn:

1. Cross-Validation: "Test-driving" your model multiple times

2. Bootstrap: "Copying and shuffling" your data to measure uncertainty

## Model Assessment vs. Model Selection

Before diving into the methods, it is important to distinguish between two different goals:

1. **Model Assessment**: The process of evaluating how well a specific model performs. We want to estimate the test error (how the model performs on data it has never seen before).

2. **Model Selection**: The process of choosing the right level of "flexibility" or complexity for a model (e.g., choosing between a simple straight line or a complex curved line).



#Cross-Validation

## 1. Validation Set Approach

### 1.1 What It Is

This is the simplest form of cross-validation. You take your data and split it into two piles:

* **Training set** (used to fit the model)
* **Validation set** (used to evaluate performance)

### 1.2 Workflow

1. Split data (e.g. 50% / 50%)
2. Train model on training set
3. Compute error on validation set


### 1.3 Error Metric (Regression)

$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

**Where:**

* $y_i$ = true value
* $\hat{y}_i$ = predicted value
* $n$ = number of validation observations

---

### 1.4 Example (Intuition)

Predict car mileage using horsepower:

* Linear model → underfits
* Quadratic model → better
* Cubic model → no improvement

Validation MSE helps us choose the **right model complexity**.

---

### 1.5 Drawbacks

1. **High variability** (depends on random split): The test error estimate can change drastically depending on which specific observations end up in the training set versus the validation set.

2. **Less training data** (Overestimating Error) $\rightarrow$ worse models: Since the model is trained on fewer observations than the full dataset, it might perform worse than a model trained on everything. This often leads to an "overestimate" of the true test error.

---


## 2. Leave-One-Out Cross-Validation (LOOCV)

LOOCV attempts to fix the drawbacks of the simple validation split.
### 2.1 Core Idea

If you have $n$ observations, you train the model on $n-1$ data points and test it on the single point you left out. You repeat this $n$ times, leaving out a different data point each time.


---

### 2.2 LOOCV Error Formula

The LOOCV estimate for the test Mean Squared Error (MSE) is the average of all these individual test errors

$$
\text{CV}(n) = \frac{1}{n} \sum_{i=1}^{n} \text{MSE}_i
$$

**Where:**

* $n$: The total number of observations in  dataset.
* $i$: The current iteration (from 1 to $n$).
* $MSE_i$: The Mean Squared Error calculated for the $i^{th}$ observation that was left out, calculated as $(y_i - \hat{y}_i)^2$.
* $CV_{(n)}$: The final estimated test error.


---

### 2.3 Why LOOCV Is Better Than Validation Set

| Aspect        | Validation Set | LOOCV    |
| ------------- | -------------- | -------- |
| Training size | ~50%           | $n-1$    |
| Bias          | High           | Very low |
| Randomness    | Yes            | No       |

---

### 2.4 Computational Shortcut (Linear Regression Only)

For least squares regression:

$$
\text{CV}(n) = \frac{1}{n} \sum_{i=1}^{n} \left( \frac{y_i - \hat{y}_i}{1 - h_i} \right)^2
$$

**Where:**

* $\hat{y}_i$ = fitted value from full model
* $h_i$ = leverage of observation $i$ (how influential it is)

> High-leverage points get **penalized more**, which corrects bias.

---

### 2.5 Limitation of LOOCV

* High Variance. Because the training sets are 99% identical, the outputs are highly correlated, which can make the average estimate less stable.
* Computational Cost. Normally, you have to fit the model $n$ times, which is slow for large data.

---



## 3. k-Fold Cross-Validation

This is the "Goldilocks" method—the middle ground that most professionals use.

### 3.1 Core Idea

1. Split data into **k equal-sized folds**
2. Use 1 fold as validation, remaining as training
3. Repeat k times
4. Average the errors

---

### 3.2 k-Fold CV Formula

$$
\text{CV}(k) = \frac{1}{k} \sum_{i=1}^{k} \text{MSE}_i
$$

**Where:**

* $k$ = number of folds
* $\text{MSE}_i$ = the error computed on the $i^{th}$ group that was held out.

---

### 3.3 Typical Choices of k

| k  | Comment           |
| -- | ----------------- |
| 5  | Good balance      |
| 10 | Industry standard |
| n  | LOOCV             |

---



## 3.4. Bias–Variance Trade-off in Cross-Validation

### Bias Perspective

k-fold has slightly more bias than LOOCV because it uses less data for training (e.g., in 5-fold, it uses 80% of the data).

* Validation set → **high bias**
* k-fold → **moderate bias**
* LOOCV → **low bias**

### Variance Perspective

k-fold has lower variance than LOOCV. This is because the training sets in k-fold have less overlap, making the results of the $k$ models less correlated.

* LOOCV → **high variance** (models are highly correlated)
* k-fold → **lower variance**

### Final Recommendation

> Use **5-fold or 10-fold CV** in practice. As it provides a test error estimate that isn't too biased but is also very stable (low variance).

---



## 4. Cross-Validation for Classification

So far, we used MSE for regression (predicting a number like "Charges"). But what if we are predicting a category (like "Smoker" or "Non-Smoker")?

In classification, we don't use MSE. Instead, we use the Error Rate (the number of misclassified observations).

### Key Change

* Regression → use **MSE**
* Classification → use **error rate (misclassification error)**

**For instance, in the classification setting, the LOOCV error rate takes the form**:

$$
\text{CV}(n) = \frac{1}{n} \sum_{i=1}^{n} \text{Err}_i
$$

**Where:**

* $\text{Err}_i = I(y_i \neq \hat{y}_i)$
* $I(\cdot)$ = indicator function (1 if true, 0 otherwise)
That is $Err_i$  an indicator ($0$ or $1$). It is $1$ if the model predicted the wrong category for observation $i$, and $0$ if it was correct

---

### Key Insight for Selection

When we plot the Training Error vs. the Cross-Validation Error, the Training Error usually keeps going down as the model gets more complex (flexibility). However, the CV Error usually forms a U-shape. The bottom of that "U" tells us the optimal level of complexity to avoid overfitting.


## **Logistic Regression & Increasing Complexity**

In real-world data science, we often don't know if the relationship between our variables is a simple straight line or a complex curve. We use **Polynomials** to add flexibility to our models.

### **1. What is "Increasing Complexity"?**

Think of it like learning to recognize animals:

* Simple model: "If it has 4 legs, it's a dog" (linear boundary)

* Medium complexity: "If it has 4 legs AND barks, it's a dog" (curved boundary)

* High complexity: "If it has 4 legs, barks, wags tail, eats bones..." (very complex boundary)

Complexity refers to how "wiggly" or flexible the model is.

* **Low Complexity (Degree 1):** A straight line (Linear). It might be too simple to catch the trend (**Underfitting**).
* **Medium Complexity (Degree 2 or 3):** A smooth curve (Quadratic/Cubic).
* **High Complexity (Degree 10+):** A very wiggly line that tries to touch every single data point (**Overfitting**). It learns the "noise" of the data rather than the actual pattern.

---

### **2. The Quadratic Logistic Regression Formula**

As complexity increases, we add squared or cubed versions of our predictors (X) to the equation. For example, a quadratic logistic regression looks like this:

$$log\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1X_1 + \beta_2X_1^2 + \beta_3X_2 + \beta_4X_2^2$$

**Variable Explanation:**

* p: The probability of an event happening (e.g., passing the exam).


* $log\left(\frac{p}{1-p}\right)$: The **Log-Odds** (or Logit), which is the standard output for logistic regression.


* $X_1, X_2$: Your original predictor variables (e.g., Age or BMI).


* $X_1^2, X_2^2$: The "complexity" terms that allow the decision boundary to be **curved**.


* $\beta$: The coefficients the model learns to weigh each variable.



---

### **3. Using 10-Fold CV to Choose Complexity**

How do we know which degree to use? We look at the **Error Rate Curves**.

* **Training Error (Blue Line):** This almost always goes **down** as complexity increases because the model "memorizes" the training data better and better.


* **Test/CV Error (Black/Brown Line):** This typically forms a **U-Shape**.


* At first, the error drops as the model gets better.
* After a certain point, the error starts to go back **up** because the model is now overfitting.





**The Goal:** We pick the degree where the **10-fold CV error is at its lowest point**. Even if the training error is lower elsewhere, the CV error is a much better guess for how the model will perform on new data.


## Practical Takeaways (Exam + Interview Ready)

* Training error is misleading
* Validation set is simple but unstable
* LOOCV is unbiased but expensive
* 5- or 10-fold CV is the best practical choice
* CV works for **both regression and classification**

---





#Bootstrap


##What Is the Bootstrap?

**Bootstrap** is a statistical technique used to **measure uncertainty**.

In simple terms:

> The bootstrap tells us **how reliable or unstable our estimate is**
> (for example: a mean, a regression coefficient, or a model parameter).

It answers questions like:

* *How much could this estimate change if I collected new data?*
* *Is this number stable or noisy?*

##Why Do We Need the Bootstrap?

In real life:

* We usually have **only one dataset**
* We **cannot repeatedly collect new samples** from the population

But to measure uncertainty, we *wish* we had many datasets.

👉 **Bootstrap solves this problem by reusing the same dataset intelligently.**

##Sampling “With Replacement” (Very Important)

### What does *with replacement* mean?

When we create a bootstrap sample:

* We randomly pick observations
* **After picking one, we put it back**
* So the same observation **can appear multiple times**

### Example

Original data ($n = 5$):

```
[2, 4, 6, 8, 10]
```

One bootstrap sample:

```
[4, 4, 10, 2, 8]
```

Another bootstrap sample:

```
[6, 6, 6, 8, 2]
```

Each bootstrap sample:

* Has the **same size ($n$)**
* Contains **duplicates**
* Misses some original points

##Bootstrap Algorithm (Step-by-Step)

Assume:

* Original dataset = ($Z$)
* Sample size = ($n$)

### Step 1: Create Bootstrap Samples

* Randomly sample ($n$) observations **with replacement**
* Call this new dataset ($Z_1^*$)

### Step 2: Compute the Estimate

* Calculate the statistic of interest
  (mean, coefficient, alpha, accuracy, etc.)
* Call it ($\hat{\theta}_1^*$)

### Step 3: Repeat

* Repeat steps 1 and 2 **B times** (usually 1000 or more)

You now have:
$$
\hat{\theta}_1^*, \hat{\theta}_2^*, \dots, \hat{\theta}_B^*
$$

---

##What Do We Get from Bootstrap?

From these repeated estimates, we can compute:

### Standard Error (Most Common Use)

**Bootstrap Standard Error:**

$$
SE_{boot}(\hat{\theta}) =
\sqrt{
\frac{1}{B-1}
\sum_{r=1}^{B}
\left(
\hat{\theta}_r^* - \bar{\hat{\theta}}^*
\right)^2
}
$$

### Meaning of Symbols

* $B$: number of bootstrap samples created
* $\hat{\theta}_r^*$: The estimate (like mean or a slope) calculated from the $r^{th}$ bootstrap dataset
* $\bar{\hat{\theta}}^*$: average of all bootstrap estimates

👉 This tells us **how much the estimate varies**.


---

##What the Bootstrap Is Used For

Bootstrap is especially useful when:

* Standard formulas for uncertainty **do not exist**
* Models are complex
* Data is limited

### Common Applications

* Standard error of regression coefficients
* Confidence intervals
* Stability of ML models
* Medians, percentiles, custom statistics

---

##hat Bootstrap Is NOT Used For

❌ It is **not** mainly for:

* Model selection
* Estimating test error

👉 That is the role of **cross-validation**

---

## 🔟 One-Sentence Summary (Exam-Ready)

> **Bootstrap estimates uncertainty by repeatedly resampling the original dataset with replacement and measuring how much the estimate varies.**

